# Building Applications with LLM APIs
The following will demonstrate how to quickly build your own application using the Google Gemini API, including basic text generation as well as multimodal image and video input.

## Import and Setup
Remember to fill in your **Gemini API Key** in this block

In [ ]:
!pip install google-generativeai gradio

In [28]:
import google.generativeai as genai
import gradio as gr
import time

# Set up Gemini API key
GOOGLE_API_KEY="<Your-API-Key>"
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

# Check if you have set your Gemini API successfully
# You should see "Set Gemini API sucessfully!!" if nothing goes wrong.
try:
    model.generate_content("test")
    print("Set Gemini API sucessfully!!")
except:
    print("There seems to be something wrong with your Gemini API.")

Set Gemini API sucessfully!!


## Text Generation

In [33]:
# construct the Gradio UI interface
with gr.Blocks() as demo:
    gr.Markdown("# Generate a Story\nUsing Gemini 1.5 Flash - Multimodal model")
    
    # story character input
    character_name = gr.Textbox(label="Enter character name:", value='Mittens')
    character_type = gr.Textbox(label="What type of character is it?", value='Cat')
    character_persona = gr.Textbox(label="What personality does the character have?", value='Mitten is a very friendly cat.')
    character_location = gr.Textbox(label="Where does the character live?", value="Andromeda Galaxy")

    # story length and premise
    length_of_story = gr.Radio(["Short","Long"], value="Short", label="Select the length of the story:")
    story_premise = gr.Dropdown(["Love","Adventure","Mystery","Horror","Comedy","Sci-Fi","Fantasy","Thriller"],
                                value=["Love","Adventure"], multiselect=True,
                                label="What is the story premise? (can select multiple)")
    temperature = gr.Slider(0.0, 1.0, 0.7, step = 0.1, label="Creativity Level (temperature):")

    def generate_story(args):
        prompt = f"""Write a {args[length_of_story]} story based on the following premise: \n
        character_name: {args[character_name]} \n
        character_type: {args[character_type]} \n
        character_persona: {args[character_persona]} \n
        character_location: {args[character_location]} \n
        story_premise: {",".join(args[story_premise])} \n
        If the story is "short", then make sure to have 5 chapters or else if it is "long" then 10 chapters. 
        Important point is that each chapter should be generated based on the premise given above.
        First start by giving the book introduction, chapter introductions and then each chapter. It should also have a proper ending.
        The book should have a prologue and an epilogue.
        """
        response = model.generate_content(prompt, generation_config=genai.types.GenerationConfig(temperature=args[temperature]))
        return [(prompt, response.text)]
    
    # submit and show result
    button = gr.Button("Submit")
    chatbot = gr.Chatbot()
    button.click(generate_story, 
                 inputs={character_name, character_type, character_persona, character_location, 
                         length_of_story, story_premise, temperature}, 
                 outputs=[chatbot])
    
demo.launch()

Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


## Image Recognition

In [52]:
file_path = 'math_equation.jpg'

def generate_answer(image, prompt):
    prefix = """
    Follow the instructions. Use a table with a row for each instruction and its result.
    INSTRUCTIONS:
    """
    prompt = prefix + prompt
    response = model.generate_content([image, prompt], generation_config=genai.types.GenerationConfig())
    return [(prompt, response.text)]
    
# construct the Gradio UI interface
with gr.Blocks() as demo:
    gr.Markdown("# Math Reasoning\nUsing Gemini 1.5 Flash - Multimodal model")
    image = gr.Image(file_path, type='pil')
    prompt = gr.Textbox(label="Ask questions about the math equation as follows:", value="""
    - Extract the formula.
    - What is the symbol right before Pi? What does it mean?
    - Is this a famous formula? Does it have a name?
    """)
    button = gr.Button("Submit")
    chatbot = gr.Chatbot()
    button.click(generate_answer, inputs=[image, prompt], outputs=[chatbot])
    
demo.launch()

Running on local URL:  http://127.0.0.1:7892

To create a public link, set `share=True` in `launch()`.


## Video Analysis

In [60]:
file_path = 'bus.mp4'

def generate_location(video_path, prompt):
    video = genai.upload_file(video_path)
    
    # check whether the file is ready to be used
    while video.state.name == "PROCESSING":
        time.sleep(10)
        video = genai.get_file(video.name)
        
    if video.state.name == "FAILED":
      raise ValueError(video.state.name)
        
    prefix = "Answer the following questions using the video only:\n"
    prompt = prefix + prompt
    response = model.generate_content([video, prompt], generation_config=genai.types.GenerationConfig())
    return [(prompt, response.text)]
    
# construct the Gradio UI interface
with gr.Blocks() as demo:
    gr.Markdown("# Video Geolocation\nUsing Gemini 1.5 Flash - Multimodal model")
    video_path = gr.Video(file_path)
    prompt = gr.Textbox(label="Answer the following questions from the video:", value="""
    - What is this video about?
    - How do you know which city it is?
    - What street is this?
    - What is the nearest intersection?
    """)
    button = gr.Button("Submit")
    chatbot = gr.Chatbot()
    button.click(generate_location, inputs=[video_path, prompt], outputs=[chatbot])
    
demo.launch()

Running on local URL:  http://127.0.0.1:7900

To create a public link, set `share=True` in `launch()`.
